In [32]:
import requests
from bs4 import BeautifulSoup

HOLOLIVE_TALENT_MAIN: str = "https://hololive.hololivepro.com/en/talents"

In [33]:
def getSoup(url: str) -> BeautifulSoup:
    r = requests.get(url)

    return BeautifulSoup(r.content, "html.parser")

In [34]:
soup = getSoup(HOLOLIVE_TALENT_MAIN)
talent_list = soup.find("ul", class_="talent_list clearfix")
talent_list_item = talent_list.find_all("li")
talent_list_item

[<li>
 <a href="https://hololive.hololivepro.com/en/talents/tokino-sora/">
 <figure>
 <img alt="" class="attachment-large size-large wp-post-image" decoding="async" fetchpriority="high" height="340" sizes="(max-width: 340px) 100vw, 340px" src="https://hololive.hololivepro.com/wp-content/uploads/2021/05/tokino_sora_thumb.png" srcset="https://hololive.hololivepro.com/wp-content/uploads/2021/05/tokino_sora_thumb.png 340w, https://hololive.hololivepro.com/wp-content/uploads/2021/05/tokino_sora_thumb-300x300.png 300w, https://hololive.hololivepro.com/wp-content/uploads/2021/05/tokino_sora_thumb-150x150.png 150w" width="340"/></figure>
 <h3>
 Tokino Sora<span>ときのそら</span>
 </h3>
 </a>
 </li>,
 <li>
 <a href="https://hololive.hololivepro.com/en/talents/roboco-san/">
 <figure>
 <img alt="" class="attachment-large size-large wp-post-image" decoding="async" height="340" sizes="(max-width: 340px) 100vw, 340px" src="https://hololive.hololivepro.com/wp-content/uploads/2020/06/roboco-san_thumb.png" 

In [35]:
talent_link = [link.find("a").get("href") for link in talent_list_item]
talent_link

['https://hololive.hololivepro.com/en/talents/tokino-sora/',
 'https://hololive.hololivepro.com/en/talents/roboco-san/',
 'https://hololive.hololivepro.com/en/talents/aki-rosenthal/',
 'https://hololive.hololivepro.com/en/talents/akai-haato/',
 'https://hololive.hololivepro.com/en/talents/shirakami-fubuki/',
 'https://hololive.hololivepro.com/en/talents/natsuiro-matsuri/',
 'https://hololive.hololivepro.com/en/talents/minato-aqua/',
 'https://hololive.hololivepro.com/en/talents/murasaki-shion/',
 'https://hololive.hololivepro.com/en/talents/nakiri-ayame/',
 'https://hololive.hololivepro.com/en/talents/yuzuki-choco/',
 'https://hololive.hololivepro.com/en/talents/oozora-subaru/',
 'https://hololive.hololivepro.com/en/talents/azki/',
 'https://hololive.hololivepro.com/en/talents/ookami-mio/',
 'https://hololive.hololivepro.com/en/talents/sakuramiko/',
 'https://hololive.hololivepro.com/en/talents/nekomata-okayu/',
 'https://hololive.hololivepro.com/en/talents/inugami-korone/',
 'https://

In [38]:
soraSoup: BeautifulSoup = getSoup(talent_link[0])
soraSoup


<!DOCTYPE html>

<html lang="en">
<head>
<!-- Global site tag (gtag.js) - Google Analytics -->
<script async="" src="https://www.googletagmanager.com/gtag/js?id=G-ZY0CGN7WFD"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());
  gtag('config', 'G-ZY0CGN7WFD');
</script>
<meta charset="utf-8"/>
<link href="https://hololive.hololivepro.com/en/talents/tokino-sora/" rel="canonical"/>
<meta content="width=device-width,initial-scale=1.0" name="viewport"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<!--en-->
<meta content="" name="description"/>
<!--en-->
<!--en-->
<title>Tokino Sora | TALENT | hololive official website</title>
<!--en-->
<!-- Facebook OGP -->
<!--en-->
<meta content="Tokino Sora | TALENT | hololive official website" property="og:title"/>
<!--en-->
<meta content="https://hololive.hololivepro.com/en/talents/tokino-sora/" property="og:url"/>
<!--en-->
<meta content="" property="og:de

In [40]:
def getName(soup: BeautifulSoup) -> dict:
    result: dict = {}
    talent_article = soup.find("article", class_="in_talent single")
    talent_name = talent_article.find("div", class_="talent_top").find("h1")

    # For some reason, the simple talent_name.get_text(strip=True) returns both the English and Japanese names concatenated
    # r"^[A-Za-z\s]+&" did not work either
    talent_name_en = [char for char in talent_name.get_text(strip=True) if "A" <= char <= "Z" or "a" <= char <= "z" or char == " "]

    result["English Name"] = "".join(talent_name_en)
    result["Japanese Name"] = talent_name.find("span").get_text(strip=True)

    return result

getName(soraSoup)

{'English Name': 'Tokino Sora', 'Japanese Name': 'ときのそら'}

In [41]:
def getLinks(soup: BeautifulSoup) -> dict:
    result: dict = {}
    talent_article = soup.find("article", class_="in_talent single")
    
    external_links_list = talent_article.find("ul", class_="t_sns clearfix")
    external_links_list_item = [link.find("a").get("href") for link in external_links_list.find_all("li")]

    result["Youtube"] = external_links_list_item[0].split('?', 1)[0]
    result["Twitter/X"] = external_links_list_item[1]

    return result

getLinks(soraSoup)

{'Youtube': 'https://www.youtube.com/channel/UCp6993wxpyDPHUpavwDFqgg',
 'Twitter/X': 'https://twitter.com/tokino_sora'}

In [42]:
def getData(soup: BeautifulSoup) -> dict:
    result: dict = {}
    talent_article = soup.find("article", class_="in_talent single")

    talent_data_box = talent_article.find("div", class_="talent_data").find("div", class_="table_box")
    talent_data_list = talent_data_box.find_all("dl")
    
    for dl in talent_data_list:
        dt = dl.find("dt").get_text(strip=True)
        dd = dl.find("dd").get_text(strip=True)
        result[dt] = dd

        if dt == "Illustrator":
            illust_link = dl.find("a").get("href")
            result["Illustrator Link"] = illust_link

    return result

getData(soraSoup)

{'Birthday': 'May 15',
 'Debut Stream': 'September 7, 2017',
 'Height': '160 cm',
 'Unit': 'hololive Generation 0',
 'Illustrator': 'Ordan',
 'Illustrator Link': 'https://twitter.com/ordan',
 'Dream': 'To perform a solo concert in Yokohama',
 'Fan Name': 'Sora-tomo (Sora’s Pals)',
 'Hashtags': 'Stream Tags: #ときのそら生放送#ときのそら実況するってよFan Art: #そらArt',
 'Catchphrases': '“Sora-tomo,” or “Sora’s Pals,” a name given to her fans.“Nun-nun,” a sound she makes when agreeing and nodding along or to pump herself up.Appears frequently in stream chat along with the emoji(๑╹ᆺ╹)',
 'Regular/Specialty Streams': 'Singing streams and chill gaming',
 'Hobbies': 'Karaoke, collecting temple stamps, rhythm games',
 'Likes': 'Sweets, Chinese food, cats, Vocaloid',
 'Special Skills': 'Clearing horror games without batting an eye'}

In [43]:
def getInfo(url: str) -> dict:
    soup = getSoup(url)

    name = getName(soup)
    links = getLinks(soup)
    data = getData(soup)

    return name | links | data

getInfo(talent_link[0])

{'English Name': 'Tokino Sora',
 'Japanese Name': 'ときのそら',
 'Youtube': 'https://www.youtube.com/channel/UCp6993wxpyDPHUpavwDFqgg',
 'Twitter/X': 'https://twitter.com/tokino_sora',
 'Birthday': 'May 15',
 'Debut Stream': 'September 7, 2017',
 'Height': '160 cm',
 'Unit': 'hololive Generation 0',
 'Illustrator': 'Ordan',
 'Illustrator Link': 'https://twitter.com/ordan',
 'Dream': 'To perform a solo concert in Yokohama',
 'Fan Name': 'Sora-tomo (Sora’s Pals)',
 'Hashtags': 'Stream Tags: #ときのそら生放送#ときのそら実況するってよFan Art: #そらArt',
 'Catchphrases': '“Sora-tomo,” or “Sora’s Pals,” a name given to her fans.“Nun-nun,” a sound she makes when agreeing and nodding along or to pump herself up.Appears frequently in stream chat along with the emoji(๑╹ᆺ╹)',
 'Regular/Specialty Streams': 'Singing streams and chill gaming',
 'Hobbies': 'Karaoke, collecting temple stamps, rhythm games',
 'Likes': 'Sweets, Chinese food, cats, Vocaloid',
 'Special Skills': 'Clearing horror games without batting an eye'}